#### Install Libraries

In [ ]:
!pip install --q streamlit ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.8/872.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.1 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 4s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


#### Modules

In [ ]:
%%writefile app.py

import streamlit as st
import os
import glob
import zipfile
import yaml
import shutil
import random
import cv2
from datetime import datetime
import time
from ultralytics import YOLO
from io import StringIO
import numpy as np
import pandas as pd
import torch
from torchvision.ops import nms
from tensorflow.keras.preprocessing.image import apply_affine_transform

YOLO_LOGO_PATH = './Gemini_Generated_Image.jpg'
col1, col2, col3 = st.columns([1, 2, 1])
with col2:
    st.image(YOLO_LOGO_PATH, use_column_width=False, width=240)

st.title("YOLO Model Training and Inference")
st.write("This app allows you to train YOLO models or run inference using your dataset. Select an option below to get started.")
st.markdown("---")
option = st.selectbox('What would you like to do today?',('Select Option', 'Train YOLO Model', 'Run Inference', 'Annotation Adjuster'))

if option == 'Train YOLO Model':
    st.header('YOLO Model Training Phase')
    st.write("Welcome to the YOLO model training phase. Please follow the steps below to set up your training.")

    def remove_directory(directory_path):

        if os.path.exists(directory_path):
            try:
                shutil.rmtree(directory_path)
            except Exception as e:
                print(f"Error: {e}")
        else:
            pass

    def update_yaml_file_paths():

        with open('./session_files/uploaded_yaml.yaml', 'r') as file:
            data = yaml.safe_load(file)

        data['train'] = './train/images'
        data['val'] = './valid/images'
        data['test'] = './test/images'

        with open('./session_files/uploaded_yaml.yaml', 'w') as file:
            yaml.safe_dump(data, file, default_flow_style=False)

    def validate_yolo_model_file(pt_file_path):
        try:
            model = YOLO(pt_file_path)
            st.success("The uploaded .pt file is a valid YOLO model.")
        except Exception as e:
            st.error("The uploaded .pt file is not a valid YOLO model")

    parent_directory = './session_files'
    remove_directory(parent_directory)
    os.makedirs(parent_directory, exist_ok=True)
    progress_tracker = ["1. Select Model Or Uplaod Custom Model", "2. Upload YAML", "3. Upload Images", "4. Data Manipulation", "5. Training Phase"]
    current_progress = len([step for step in progress_tracker if step]) / len(progress_tracker)
    st.progress(current_progress)
    st.subheader("Step 1: Select or Upload YOLO Model")
    model_choice = st.radio("Choose how you'd like to proceed:", ('Select Pre-trained Model', 'Upload Custom Model'))
    selected_model = None
    custom_model = False
    if model_choice == 'Select Pre-trained Model':
        selected_models = {
            'yolov8n.pt': 'YOLOv8-N (Nano) ~3.2 M Params',
            'yolov8s.pt': 'YOLOv8-S (Small) ~11.2 M Params',
            'yolov8m.pt': 'YOLOv8-M (Medium) ~25.9 M Params',
            'yolov8l.pt': 'YOLOv8-L (Large) ~43.7 M Params',
            'yolov8x.pt': 'YOLOv8-X (Extra-large) ~68.2 M Params',
            'yolov10n.pt': 'YOLOv10-N (Nano) ~2.3 M Params',
            'yolov10s.pt': 'YOLOv10-S (Small) ~7.2 M Params',
            'yolov10m.pt': 'YOLOv10-M (Medium) ~15.4 M Params',
            'yolov10l.pt': 'YOLOv10-L (Large) ~24.4 M Params',
            'yolov10x.pt': 'YOLOv10-X (Extra-large) ~29.5 M Params',
        }
        selected_model = st.selectbox('Choose a pre-trained YOLO model', list(selected_models.keys()), format_func=lambda x: selected_models[x])
        st.success(f"Using pre-trained model: {selected_model} for training.")
    elif model_choice == 'Upload Custom Model':
        st.subheader('Step 1: Upload the YOLO Model (.pt)')
        model_file = st.file_uploader("Upload YOLO .pt model file", type=['pt'], help="Upload the YOLO model (.pt format) to run inference or continue training.")
        if model_file:
            custom_model = True
            with open('./session_files/custom_model.pt', "wb") as f:
                f.write(model_file.getbuffer())
            validate_yolo_model_file('./session_files/custom_model.pt')
        else:
            st.warning("Please upload your YOLO model (.pt) file to continue.")
    st.subheader("Step 2: Upload the YAML File")
    yaml_file = st.file_uploader("Upload YAML file", type=['yaml', 'yml'])
    dataset_directory = None

    if yaml_file:
        with open(file='./session_files/uploaded_yaml.yaml', mode="wb") as f:
            f.write(yaml_file.getbuffer())
        update_yaml_file_paths()
        stringio = StringIO(yaml_file.getvalue().decode("utf-8"))
        yaml_content = yaml.safe_load(stringio)

        if all(key in yaml_content for key in ['train', 'val', 'test', 'nc', 'names']):
            st.success("YAML file is valid.")
            st.write(f"Number of classes: {yaml_content['nc']}")
            st.write(f"Classes: {', '.join(yaml_content['names'])}")
        else:
            st.error("YAML file missing required fields.")

    st.header('Step 3: Upload Training, Validation, and Testing Images')
    image_zip = st.file_uploader("Upload image zip file", type=['zip'])
    dataset_directory = os.path.join(os.getcwd(), 'session_files')

    def check_folder_structure(extracted_folder):
        required_structure = {
            "train": ["images", "labels"],
            "valid": ["images", "labels"],
            "test": ["images", "labels"]
        }
        structure_status = {}
        for main_folder, subfolders in required_structure.items():
            main_folder_path = os.path.join(extracted_folder, main_folder)
            if not os.path.exists(main_folder_path):
                structure_status[main_folder] = False
            else:
                structure_status[main_folder] = all(os.path.exists(os.path.join(main_folder_path, subfolder)) for subfolder in subfolders)
        return structure_status

    def display_structure_result(structure_status):
        st.write("### Your Uploaded Structure:")
        for folder, status in structure_status.items():
            symbol = "✅" if status else "❌"
            st.write(f"{symbol} {folder}/")

            if status:
                st.write(f"  - {folder}/images/")
                st.write(f"  - {folder}/labels/")
        st.write("### Correct Structure:")
        st.write("""
        - train/
            - images/
            - labels/
        - valid/
            - images/
            - labels/
        - test/
            - images/
            - labels/
        """)

    if image_zip is not None:
        temp_extract_dir = './session_files/test_structure/'
        with zipfile.ZipFile(image_zip, 'r') as zip_ref:
            zip_ref.extractall(temp_extract_dir)
        st.success("Images unzipped successfully!")
        structure_status = check_folder_structure(temp_extract_dir)
        if all(structure_status.values()):
            for item in os.listdir(temp_extract_dir):
                shutil.move(os.path.join(temp_extract_dir, item), os.path.join(parent_directory, item))
            shutil.rmtree(temp_extract_dir)
            valid_extensions = ('.jpg', '.jpeg', '.png')
            for path1 in ['/train/images', '/valid/images', '/test/images']:
                for filename in os.path.join(parent_directory, path1):
                    if not filename.lower().endswith(valid_extensions):
                        try:
                            os.remove(os.path.join(parent_directory, path1, filename))
                        except Exception as e:
                            continue
            train_images = os.listdir(os.path.join(dataset_directory, 'train/images'))
            train_count = len(train_images)
            test_images = os.listdir(os.path.join(dataset_directory, 'test/images'))
            test_count = len(test_images)
            val_images = os.listdir(os.path.join(dataset_directory, 'valid/images'))
            val_count = len(val_images)
            st.write(f"Train images: {train_count}")
            st.write(f"Validation images: {val_count}")
            st.write(f"Test images: {test_count}")
            for idx,filename in enumerate(train_images):
              if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                sample_image_path = filename
                break
            sample_image = cv2.imread(os.path.join(dataset_directory, 'train/images', sample_image_path))
            sample_image_rgb = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
        else:
            st.error("The uploaded zip file does not have the correct structure. Please check the details below:")
            display_structure_result(structure_status)
            shutil.rmtree(temp_extract_dir)
    else:
        train_count = 0
        st.write("No images uploaded yet.")

    st.header('Step 4: Data Manipulation')
    if 'selected_techniques' not in st.session_state:
        st.session_state.selected_techniques = {}
    if 'try_technique' not in st.session_state:
        st.session_state.try_technique = None
    if 'technique_values' not in st.session_state:
        st.session_state.technique_values = []
    data_manipulation = st.radio("Do you want to apply data manipulation?", ["No", "Yes"], index=0)

    def apply_manipulation(image, manipulation, value):
        if manipulation == 'Add Noise':
            noise = np.random.normal(loc=0, scale=value/100, size=image.shape).astype('uint8')
            return cv2.add(image, noise)
        elif manipulation == 'Brightness':
            hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            hsv[:, :, 2] = cv2.add(hsv[:, :, 2], value)
            return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        elif manipulation == 'Contrast':
            return cv2.convertScaleAbs(image, alpha=1 + value/100, beta=0)
        elif manipulation == 'Flip Horizontally':
            return cv2.flip(image, 1) if value > 0 else image
        elif manipulation == 'Flip Vertically':
            return cv2.flip(image, 0) if value > 0 else image
        elif manipulation == 'Rotate':
            return apply_affine_transform(image, theta=value)
        elif manipulation == 'Zoom':
            return apply_affine_transform(image, zx=1 + value/100, zy=1 + value/100)
        elif manipulation == 'Shear':
            return apply_affine_transform(image, shear=value)

    manipulations = {
        'Add Noise': (0, 100, 0),
        'Brightness': (0, 100, 0),
        'Contrast': (0, 100, 0),
        'Flip Horizontally': (0, 360, 0),
        'Flip Vertically': (0, 360, 0),
        'Rotate': (0, 360, 0),
        'Zoom': (0, 100, 0),
        'Shear': (0, 100, 0),
    }

    if data_manipulation == "Yes" and train_count > 0:
        st.subheader("Select and Try Manipulation Techniques")
        available_techniques = {k: v for k, v in manipulations.items() if k not in st.session_state.selected_techniques}
        manipulation_choice = st.selectbox("Choose a technique to apply:", list(available_techniques.keys()), index=0)
        if manipulation_choice:
            st.subheader(f"Testing {manipulation_choice}")
            min_val, max_val, default_val = manipulations[manipulation_choice]
            slider_value = st.slider(f"Adjust {manipulation_choice}:", min_value=min_val, max_value=max_val, value=default_val)
            manipulated_image = apply_manipulation(sample_image_rgb, manipulation_choice, slider_value)
            st.image(manipulated_image, caption=f"Preview: {manipulation_choice} with value {slider_value}", use_column_width=True)
            if st.button("Add Technique", key="add_technique"):
                st.session_state.selected_techniques[manipulation_choice] = slider_value
                st.session_state.technique_values.append({'Technique': manipulation_choice, 'Value': slider_value})
        if st.session_state.technique_values:
            st.subheader("Added Techniques")
            df_transformations = pd.DataFrame(st.session_state.technique_values)
            st.table(df_transformations)
            if st.button("Apply Techniques to Training Data"):
                train_images_path = './session_files/train/images/'
                train_labels_path = './session_files/train/labels/'
                valid_extensions = ('.jpg', '.jpeg', '.png')
                for img_name in os.listdir(train_images_path):
                    if img_name.lower().endswith(valid_extensions):
                        img_path = os.path.join(train_images_path, img_name)
                        img = cv2.imread(img_path)
                        for technique, value in st.session_state.selected_techniques.items():
                            manipulated_img = apply_manipulation(img, technique, value)
                            new_img_name = f"{os.path.splitext(img_name)[0]}_{technique.lower()}.png"
                            new_img_path = os.path.join(train_images_path, new_img_name)
                            cv2.imwrite(new_img_path, manipulated_img)
                            label_name = os.path.splitext(img_name)[0] + '.txt'
                            new_label_name = f"{os.path.splitext(img_name)[0]}_{technique.lower()}.txt"
                            if os.path.exists(os.path.join(train_labels_path, label_name)):
                                shutil.copy(os.path.join(train_labels_path, label_name), os.path.join(train_labels_path, new_label_name))
                current_date = datetime.now().strftime("%Y%m%d")
                if custom_model:
                    model_base_name = model_file.name.split('.')[0]
                else:
                    model_base_name = selected_model.split('.')[0]
                csv_filename = f"{model_base_name}_transformation_log_{current_date}.csv"
                df_transformations.to_csv(path_or_buf=f'./session_files/{csv_filename}', header=True, encoding='utf-8', mode='w', index=False)
                st.success(f"All selected techniques have been applied to the training data.")
                st.success(f"Train images: {((df_transformations.shape[0]*train_count) + train_count)}")
    else:
        st.write(f"Train images: {train_count}")

    st.header('Step 5: Train the Model')
    epochs = st.number_input('Enter number of epochs for training:', min_value=1, max_value=100, value=1, step=1)
    if epochs > 10:
        st.warning("You have selected more than 10 epochs. Training may take a long time depending on the model.", icon="⚠️")
    if st.button("Train"):
        st.write("Training in progress...")
        try:
            if torch.cuda.is_available():
                st.warning("Training will be done on GPU")
                device = torch.device('cuda')
            else:
                st.warning("Training will be done on CPU")
                device = torch.device('cpu')
            start_time = time.time()
            if not custom_model:
                model = YOLO(selected_model).to(device)
            else:
                model = YOLO('./session_files/custom_model.pt').to(device)
            model.train(data=os.path.join(os.getcwd(), 'session_files', 'uploaded_yaml.yaml'), epochs=epochs, device=device)
            total_time = time.time() - start_time
            total_time_minutes = total_time / 60
            total_time_hours = total_time / 3600
            st.success(f"Training completed successfully in ~{total_time_minutes:.2f} minutes / ~{total_time_hours:.2f} hours.")
            current_date = datetime.now().strftime("%Y%m%d")
            trained_model_folder = f"./session_files/TrainedModel_{current_date}"
            if not os.path.exists(trained_model_folder):
                os.makedirs(trained_model_folder)
            if custom_model:
                model_filename = f"{model_file.name.split('.')[0]}_epochs{epochs}.pt"
            else:
                model_filename = f"{selected_model.split('.')[0]}_epochs{epochs}.pt"
            trained_model_path = os.path.join(trained_model_folder, model_filename)
            model.save(trained_model_path)
            if os.path.exists('/content/runs'):
                shutil.move('/content/runs', trained_model_folder)
            else:
                pass
            csv_file_path = glob.glob('./session_files/*_transformation_log_*.csv')
            if csv_file_path:
                shutil.move(csv_file_path[0], trained_model_folder)
            else:
                pass
            zip_filename = f"{trained_model_folder.replace('./session_files/','')}.zip"
            shutil.make_archive(trained_model_folder.replace('./session_files/',''), 'zip', trained_model_folder)
            with open(zip_filename, 'rb') as f:
                st.download_button('Download Model', f, file_name=zip_filename)
        except Exception as e:
            st.error(f"An error occurred during training: {str(e)}")

elif option == 'Run Inference':
    st.header("YOLO Model Inference Phase")
    st.write("You can run YOLO inference on uploaded images.")
    st.markdown("---")
    progress_tracker = ["1. Select Model", "2. Upload Images", "6. Run Inference"]
    current_progress = len([step for step in progress_tracker if step]) / len(progress_tracker)
    st.progress(current_progress)

    def check_test_folder_structure(extracted_folder):
        required_structure = {
            "test": ["images"]
        }
        structure_status = {}
        for main_folder, subfolders in required_structure.items():
            main_folder_path = os.path.join(extracted_folder, main_folder)
            if not os.path.exists(main_folder_path):
                structure_status[main_folder] = False
            else:
                structure_status[main_folder] = all(os.path.exists(os.path.join(main_folder_path, subfolder)) for subfolder in subfolders)
        return structure_status

    def display_test_structure_result(structure_status):
        st.write("### Your Uploaded Structure:")
        for folder, status in structure_status.items():
            symbol = "✅" if status else "❌"
            st.write(f"{symbol} {folder}/")
            if status:
                st.write(f"  - {folder}/images/")
        st.write("### Correct Structure:")
        st.write("""
        - test/
            - images/
        """)

    def remove_directory(directory_path):
        if os.path.exists(directory_path):
            try:
                shutil.rmtree(directory_path)
            except Exception as e:
                print(f"Error: {e}")
        else:
            pass

    def filter_and_merge_boxes(boxes, scores, threshold=0.7):
        scores = np.array(scores)
        indices = np.where(scores > threshold)[0]
        filtered_boxes = [boxes[i] for i in indices]
        filtered_scores = [scores[i] for i in indices]
        if len(filtered_boxes) == 0:
            return [], []
        boxes = np.array(filtered_boxes)
        scores = np.array(filtered_scores)
        boxes_tensor = torch.tensor(boxes, dtype=torch.float32)
        scores_tensor = torch.tensor(scores, dtype=torch.float32)
        keep = nms(boxes_tensor, scores_tensor, iou_threshold=0.5)
        merged_boxes = boxes[keep.numpy()].tolist()
        merged_scores = scores[keep.numpy()].tolist()
        return merged_boxes, merged_scores

    def validate_yolo_model_file(pt_file_path):
        try:
            model = YOLO(pt_file_path)
            st.success("The uploaded .pt file is a valid YOLO model.")
        except Exception as e:
            st.error("The uploaded .pt file is not a valid YOLO model")

    parent_directory = './session_files'
    remove_directory(parent_directory)
    os.makedirs(parent_directory, exist_ok=True)
    st.subheader('Step 1: Upload the YOLO Model (.pt)')
    model_file = st.file_uploader("Upload YOLO .pt model file", type=['pt'],
    help="Upload the YOLO model (.pt format) to run inference.")
    if model_file:
        with open('./session_files/custom_model.pt', "wb") as f:
            f.write(model_file.getbuffer())
        validate_yolo_model_file('./session_files/custom_model.pt')
    st.subheader('Step 2: Upload Test Images')
    test_zip = st.file_uploader("Upload test image zip", type=['zip'], help="Upload a zip file containing your test images in the correct structure (test/ folder).")
    if test_zip:
      temp_extract_dir = './session_files/test_structure/'
      with zipfile.ZipFile(test_zip, 'r') as zip_ref:
          zip_ref.extractall(temp_extract_dir)
      st.success("Images unzipped successfully!")
      structure_status = check_test_folder_structure(temp_extract_dir)
    if model_file and test_zip:
        if torch.cuda.is_available():
            st.warning("Inference will be done on GPU")
            device = torch.device('cuda')
        else:
            st.warning("Inference will be done on CPU")
            device = torch.device('cpu')
        if all(structure_status.values()):
            for item in os.listdir(temp_extract_dir):
                shutil.move(os.path.join(temp_extract_dir, item), os.path.join(parent_directory, item))
            shutil.rmtree(temp_extract_dir)
            valid_extensions = ('.jpg', '.jpeg', '.png')
            for path1 in ['/test/images']:
                for filename in os.path.join(parent_directory, path1):
                    if not filename.lower().endswith(valid_extensions):
                        try:
                            os.remove(os.path.join(parent_directory, path1, filename))
                        except Exception as e:
                            continue
            test_images_path = os.path.join(parent_directory, 'test/images')
            test_images = os.listdir(test_images_path)
            test_count = len(test_images)
            st.write(f"Number of test images: {test_count}")
            for idx,filename in enumerate(test_images):
              if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                sample_image_path = filename
                break
            sample_image = cv2.imread(os.path.join(test_images_path, sample_image_path))
            sample_image_rgb = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
            st.subheader("Adjust Bounding Box Parameters for Sample Image")
            thickness = st.slider("Select bounding box thickness", min_value=1, max_value=10, value=2, step=1, help="Choose the thickness of the bounding box lines (1-10).")
            font_scale = st.slider("Select font scale", min_value=0.5, max_value=2.0, value=0.6, step=0.1, help="Set the font size for the labels displayed on the bounding boxes.")
            threshold = st.slider("Set threshold", min_value=0, max_value=100, value=70, step=1, help="Set the confidence threshold (0-100%) for detecting objects. Only objects with a confidence score above this threshold will be shown.")
            box_color_hex = st.color_picker("Select bounding box color", "#00FF00", help="Pick a color for the bounding box.")
            box_color_rgb = tuple(int(box_color_hex.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))
            st.subheader("Run Inference on Sample Image")
            if st.button("Run Sample Inference"):
                model = YOLO('./session_files/custom_model.pt').to(device)
                st.write("Running inference on the sample image...")
                results = model(sample_image,device=device)
                detections = results[0].boxes
                image_with_boxes = sample_image.copy()
                filtered_boxes = [box.xyxy[0].tolist() for box in detections]
                scores = [box.conf[0].item() for box in detections]
                merged_boxes, merged_scores = filter_and_merge_boxes(filtered_boxes, scores, threshold / 100)
                for box, score in zip(merged_boxes, merged_scores):
                    x1, y1, x2, y2 = map(int, box)
                    class_id = int(detections[0].cls[0].item())
                    label = results[0].names[class_id]
                    cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), box_color_rgb, thickness)
                    cv2.putText(image_with_boxes, f"{label} {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, box_color_rgb, thickness)
                col1, col2 = st.columns(2)
                with col1:
                    st.image(sample_image_rgb, caption="Sample Image", use_column_width=True)
                with col2:
                    st.image(image_with_boxes, caption="Predicted Image", use_column_width=True)
            st.subheader("Run Inference on All Test Images")
            if st.button("Run Inference on All Images"):
                st.write("Running inference on all test images...")
                try:
                    model = YOLO('./session_files/custom_model.pt').to(device)
                    inference_folder = './session_files/inference/'
                    if not os.path.exists(inference_folder):
                        os.makedirs(inference_folder)
                    results_list = []
                    start_time = time.time()
                    for img_name in test_images:
                        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                            img_path = os.path.join(test_images_path, img_name)
                            img = cv2.imread(img_path)
                            start_time = time.time()
                            results = model(img,device=device)
                            detections = results[0].boxes
                            image_with_boxes = img.copy()
                            filtered_boxes = [box.xyxy[0].tolist() for box in detections]
                            scores = [box.conf[0].item() for box in detections]
                            merged_boxes, merged_scores = filter_and_merge_boxes(filtered_boxes, scores, threshold / 100)
                            for box, score in zip(merged_boxes, merged_scores):
                                x1, y1, x2, y2 = map(int, box)
                                class_id = int(detections[0].cls[0].item())
                                label = results[0].names[class_id]
                                cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), box_color_rgb, thickness)
                                cv2.putText(image_with_boxes, f"{label} {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, box_color_rgb, thickness)
                                results_list.append({
                                    "ImageName": img_name,
                                    "TimeTaken": time.time() - start_time,
                                    "LabelID": class_id,
                                    "Label": label,
                                    "Score": score,
                                    "X0-Value": x1,
                                    "X1-Value": x2,
                                    "Y0-Value": y1,
                                    "Y1-Value": y2
                                })
                            cv2.imwrite(os.path.join(inference_folder, img_name), image_with_boxes)
                        else:
                            continue
                    total_time = time.time() - start_time
                    total_time_minutes = total_time / 60
                    total_time_hours = total_time / 3600
                    st.success(f"Inference on all images completed! in ~{total_time_minutes:.2f} minutes / ~{total_time_hours:.2f} hours.")
                    current_date = datetime.now().strftime("%Y%m%d")
                    csv_filename = f"inference_results_{current_date}.csv"
                    csv_path = os.path.join(inference_folder, csv_filename)
                    df:pd.DataFrame = pd.DataFrame(data=results_list)
                    df['TimeTaken']:pd.Series = df['TimeTaken'].round(6)
                    df['Score']:pd.Series = df['Score'].round(6)
                    df.to_csv(path_or_buf=csv_path, header=True, encoding='utf-8', mode='w', index=False)
                    zip_filename = f"InferenceResults_{current_date}.zip"
                    shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', inference_folder)
                    with open(zip_filename, 'rb') as f:
                        st.download_button('Download Inference Results', f, file_name=zip_filename)
                except Exception as e:
                    st.error(f"An error occurred during Inference: {str(e)}")
        else:
            if test_zip:
                st.error("Invalid structure! Please upload a zip with 'test/images' folder containing images.")
                display_test_structure_result(structure_status)


Overwriting app.py


#### Retrieve the External URL

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!cat logs.txt




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.158.238:8501



#### Start Localtunnel
[Help](https://discuss.streamlit.io/t/how-to-launch-streamlit-app-from-google-colab-notebook/42399/2)

In [ ]:
!npx localtunnel --port 8501

your url is: https://rotten-taxes-dance.loca.lt
